In [27]:
import re
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from textblob import TextBlob as tb
from contractions import CONTRACTION_MAP
stopwords = stopwords.words('english')
stopwords.remove('no')
stopwords.remove('not')
stopwords.append('sa')
stopwords.append('na')
stopwords.append('ng')
stopwords.append('ko')
stopwords.append('ba')

In [28]:
data = pd.read_csv('final_dataset.csv',index_col=None,header=0)
n = data.shape[0]

In [29]:
data.head()

,id,submission_title,submission_text,author,body,depth,parent_id,score,subreddit_name_prefixed,replies,length,labels
0,d4xury6,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",Ryvaeus,"No questions, just wanted to thank you for pos...",0,t3_4r2tl4,31,r/Philippines,4,245,supporting
1,d4yi9y9,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",Somebody_over_there,Holy shit thank you so much! \nAs someone ...,0,t3_4r2tl4,7,r/Philippines,8,393,supporting
2,d4yhm4h,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",PotatoMurderer,> people there are nice and friendly; super no...,0,t3_4r2tl4,7,r/Philippines,0,507,commenting
3,d4y5kmd,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",mazikeen24,Thanks for posting this! Have you tried Yaz or...,0,t3_4r2tl4,3,r/Philippines,16,238,supporting
4,d4y8tvk,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",a_reverse_giraffe,"As a minor, did you need to show your parents ...",0,t3_4r2tl4,3,r/Philippines,4,55,questioning


## Cleaning

In [30]:
def clean_text(df):
    """
    Function for cleaning text based on class jupyter notebook on text manipulation
    """
    text = df.copy()
    text['cleaned'] = text['body'].fillna('')
    text['cleaned'] = text['cleaned'].str.lower()
    text['cleaned'] = text['cleaned'].str.replace(r'([-])', ' ')
    text['cleaned'] = text['cleaned'].str.replace(r'([^A-Z a-z\'])|', '')
    text['cleaned'] = text['cleaned'].apply(lambda x: ' '.join([w for w in x.split() if w not in (stopwords)]))
    
    text['cleaned_sub'] = text['submission_text'].fillna('')
    text['cleaned_sub'] = text['cleaned_sub'].str.lower()
    text['cleaned_sub'] = text['cleaned_sub'].str.replace(r'([-])', ' ')
    text['cleaned_sub'] = text['cleaned_sub'].str.replace(r'([^A-Z a-z\'])|', '')
    text['cleaned_sub'] = text['cleaned_sub'].apply(lambda x: ' '.join([w for w in x.split() if w not in (stopwords)]))
    
    return text

In [31]:
clean = clean_text(data)

In [32]:
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

In [33]:
expand_list = []
for i in range(n):
    expand_list.append(expand_contractions(clean.loc[i]['cleaned']))
clean['expand'] = expand_list
clean.head()

,id,submission_title,submission_text,author,body,depth,parent_id,score,subreddit_name_prefixed,replies,length,labels,cleaned,cleaned_sub,expand
0,d4xury6,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",Ryvaeus,"No questions, just wanted to thank you for pos...",0,t3_4r2tl4,31,r/Philippines,4,245,supporting,no questions wanted thank posting topic birth ...,updated september scroll downhi lengthy post c...,no questions wanted thank posting topic birth ...
1,d4yi9y9,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",Somebody_over_there,Holy shit thank you so much! \nAs someone ...,0,t3_4r2tl4,7,r/Philippines,8,393,supporting,holy shit thank much someone recently turned s...,updated september scroll downhi lengthy post c...,holy shit thank much someone recently turned s...
2,d4yhm4h,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",PotatoMurderer,> people there are nice and friendly; super no...,0,t3_4r2tl4,7,r/Philippines,0,507,commenting,people nice friendly super non judgmental espe...,updated september scroll downhi lengthy post c...,people nice friendly super non judgmental espe...
3,d4y5kmd,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",mazikeen24,Thanks for posting this! Have you tried Yaz or...,0,t3_4r2tl4,3,r/Philippines,16,238,supporting,thanks posting tried yaz oral contraceptives p...,updated september scroll downhi lengthy post c...,thanks posting tried yaz oral contraceptives p...
4,d4y8tvk,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",a_reverse_giraffe,"As a minor, did you need to show your parents ...",0,t3_4r2tl4,3,r/Philippines,4,55,questioning,minor need show parents consent,updated september scroll downhi lengthy post c...,minor need show parents consent


In [34]:
expand_list = []
for i in range(n):
    expand_list.append(expand_contractions(clean.loc[i]['cleaned_sub']))
clean['expand_sub'] = expand_list
clean.head()

,id,submission_title,submission_text,author,body,depth,parent_id,score,subreddit_name_prefixed,replies,length,labels,cleaned,cleaned_sub,expand,expand_sub
0,d4xury6,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",Ryvaeus,"No questions, just wanted to thank you for pos...",0,t3_4r2tl4,31,r/Philippines,4,245,supporting,no questions wanted thank posting topic birth ...,updated september scroll downhi lengthy post c...,no questions wanted thank posting topic birth ...,updated september scroll downhi lengthy post c...
1,d4yi9y9,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",Somebody_over_there,Holy shit thank you so much! \nAs someone ...,0,t3_4r2tl4,7,r/Philippines,8,393,supporting,holy shit thank much someone recently turned s...,updated september scroll downhi lengthy post c...,holy shit thank much someone recently turned s...,updated september scroll downhi lengthy post c...
2,d4yhm4h,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",PotatoMurderer,> people there are nice and friendly; super no...,0,t3_4r2tl4,7,r/Philippines,0,507,commenting,people nice friendly super non judgmental espe...,updated september scroll downhi lengthy post c...,people nice friendly super non judgmental espe...,updated september scroll downhi lengthy post c...
3,d4y5kmd,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",mazikeen24,Thanks for posting this! Have you tried Yaz or...,0,t3_4r2tl4,3,r/Philippines,16,238,supporting,thanks posting tried yaz oral contraceptives p...,updated september scroll downhi lengthy post c...,thanks posting tried yaz oral contraceptives p...,updated september scroll downhi lengthy post c...
4,d4y8tvk,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",a_reverse_giraffe,"As a minor, did you need to show your parents ...",0,t3_4r2tl4,3,r/Philippines,4,55,questioning,minor need show parents consent,updated september scroll downhi lengthy post c...,minor need show parents consent,updated september scroll downhi lengthy post c...


In [37]:
# cleaning again

def clean_text_2(clean):
    text = clean.copy()
    text['expand'] = text['expand'].str.replace(r'([\'])', '')
    text['expand'] = text['expand'].apply(lambda x: ' '.join([w for w in x.split() if w not in (stopwords)]))
    
    text['expand_sub'] = text['expand_sub'].str.replace(r'([\'])', '')
    text['expand_sub'] = text['expand_sub'].apply(lambda x: ' '.join([w for w in x.split() if w not in (stopwords)]))
    return text

expand = clean_text_2(clean)
expand.head()

,id,submission_title,submission_text,author,body,depth,parent_id,score,subreddit_name_prefixed,replies,length,labels,cleaned,cleaned_sub,expand,expand_sub
0,d4xury6,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",Ryvaeus,"No questions, just wanted to thank you for pos...",0,t3_4r2tl4,31,r/Philippines,4,245,supporting,no questions wanted thank posting topic birth ...,updated september scroll downhi lengthy post c...,no questions wanted thank posting topic birth ...,updated september scroll downhi lengthy post c...
1,d4yi9y9,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",Somebody_over_there,Holy shit thank you so much! \nAs someone ...,0,t3_4r2tl4,7,r/Philippines,8,393,supporting,holy shit thank much someone recently turned s...,updated september scroll downhi lengthy post c...,holy shit thank much someone recently turned s...,updated september scroll downhi lengthy post c...
2,d4yhm4h,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",PotatoMurderer,> people there are nice and friendly; super no...,0,t3_4r2tl4,7,r/Philippines,0,507,commenting,people nice friendly super non judgmental espe...,updated september scroll downhi lengthy post c...,people nice friendly super non judgmental espe...,updated september scroll downhi lengthy post c...
3,d4y5kmd,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",mazikeen24,Thanks for posting this! Have you tried Yaz or...,0,t3_4r2tl4,3,r/Philippines,16,238,supporting,thanks posting tried yaz oral contraceptives p...,updated september scroll downhi lengthy post c...,thanks posting tried yaz oral contraceptives p...,updated september scroll downhi lengthy post c...
4,d4y8tvk,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",a_reverse_giraffe,"As a minor, did you need to show your parents ...",0,t3_4r2tl4,3,r/Philippines,4,55,questioning,minor need show parents consent,updated september scroll downhi lengthy post c...,minor need show parents consent,updated september scroll downhi lengthy post c...


## Lemmatizing

In [41]:
expand.head(5)

,id,submission_title,submission_text,author,body,depth,parent_id,score,subreddit_name_prefixed,replies,length,labels,cleaned,cleaned_sub,expand,expand_sub
0,d4xury6,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",Ryvaeus,"No questions, just wanted to thank you for pos...",0,t3_4r2tl4,31,r/Philippines,4,245,supporting,no questions wanted thank posting topic birth ...,updated september scroll downhi lengthy post c...,no questions wanted thank posting topic birth ...,updated september scroll downhi lengthy post c...
1,d4yi9y9,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",Somebody_over_there,Holy shit thank you so much! \nAs someone ...,0,t3_4r2tl4,7,r/Philippines,8,393,supporting,holy shit thank much someone recently turned s...,updated september scroll downhi lengthy post c...,holy shit thank much someone recently turned s...,updated september scroll downhi lengthy post c...
2,d4yhm4h,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",PotatoMurderer,> people there are nice and friendly; super no...,0,t3_4r2tl4,7,r/Philippines,0,507,commenting,people nice friendly super non judgmental espe...,updated september scroll downhi lengthy post c...,people nice friendly super non judgmental espe...,updated september scroll downhi lengthy post c...
3,d4y5kmd,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",mazikeen24,Thanks for posting this! Have you tried Yaz or...,0,t3_4r2tl4,3,r/Philippines,16,238,supporting,thanks posting tried yaz oral contraceptives p...,updated september scroll downhi lengthy post c...,thanks posting tried yaz oral contraceptives p...,updated september scroll downhi lengthy post c...
4,d4y8tvk,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",a_reverse_giraffe,"As a minor, did you need to show your parents ...",0,t3_4r2tl4,3,r/Philippines,4,55,questioning,minor need show parents consent,updated september scroll downhi lengthy post c...,minor need show parents consent,updated september scroll downhi lengthy post c...


In [44]:
# cleaned
def lemmatize(clean):
    to_lemm = clean.copy()
    to_lemm['lemmatized'] = ''
    for i in range(n):
        lemmatized = ''
        if( len(to_lemm['expand'][i]) > 0 ):
            blob = tb(to_lemm['expand'][i]).sentences[0]
            tags = blob.tags
            for j in range(0,len(tags)):
                if tags[j][1].startswith('NN'):
                    lemmatized += tags[j][0].lemmatize('n')
                    lemmatized += ' '
                elif tags[j][1].startswith('VB'):
                    lemmatized += tags[j][0].lemmatize('v')
                    lemmatized += ' '
                elif tags[j][1].startswith('JJ'):
                    lemmatized += tags[j][0].lemmatize('a')
                    lemmatized += ' '
                else:
                    lemmatized += tags[j][0]
                    lemmatized += ' '
        to_lemm['lemmatized'][i] = lemmatized.strip()
    return to_lemm

In [45]:
lemm = lemmatize(expand)
lemm.head()

c:\users\limri\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,id,submission_title,submission_text,author,body,depth,parent_id,score,subreddit_name_prefixed,replies,length,labels,cleaned,cleaned_sub,expand,expand_sub,lemmatized
0,d4xury6,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",Ryvaeus,"No questions, just wanted to thank you for pos...",0,t3_4r2tl4,31,r/Philippines,4,245,supporting,no questions wanted thank posting topic birth ...,updated september scroll downhi lengthy post c...,no questions wanted thank posting topic birth ...,updated september scroll downhi lengthy post c...,no question want thank post topic birth contro...
1,d4yi9y9,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",Somebody_over_there,Holy shit thank you so much! \nAs someone ...,0,t3_4r2tl4,7,r/Philippines,8,393,supporting,holy shit thank much someone recently turned s...,updated september scroll downhi lengthy post c...,holy shit thank much someone recently turned s...,updated september scroll downhi lengthy post c...,holy shit thank much someone recently turn sup...
2,d4yhm4h,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",PotatoMurderer,> people there are nice and friendly; super no...,0,t3_4r2tl4,7,r/Philippines,0,507,commenting,people nice friendly super non judgmental espe...,updated september scroll downhi lengthy post c...,people nice friendly super non judgmental espe...,updated september scroll downhi lengthy post c...,people nice friendly super non judgmental espe...
3,d4y5kmd,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",mazikeen24,Thanks for posting this! Have you tried Yaz or...,0,t3_4r2tl4,3,r/Philippines,16,238,supporting,thanks posting tried yaz oral contraceptives p...,updated september scroll downhi lengthy post c...,thanks posting tried yaz oral contraceptives p...,updated september scroll downhi lengthy post c...,thanks post tried yaz oral contraceptive prior...
4,d4y8tvk,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",a_reverse_giraffe,"As a minor, did you need to show your parents ...",0,t3_4r2tl4,3,r/Philippines,4,55,questioning,minor need show parents consent,updated september scroll downhi lengthy post c...,minor need show parents consent,updated september scroll downhi lengthy post c...,minor need show parent consent


In [46]:
# cleaned_sub
def lemmatize_2(clean):
    to_lemm = clean.copy()
    to_lemm['lemm_sub'] = ''
    for i in range(n):
        lemmatized = ''
        if( len(to_lemm['expand_sub'][i]) > 0 ):
            blob = tb(to_lemm['expand_sub'][i]).sentences[0]
            tags = blob.tags
            for j in range(0,len(tags)):
                if tags[j][1].startswith('NN'):
                    lemmatized += tags[j][0].lemmatize('n')
                    lemmatized += ' '
                elif tags[j][1].startswith('VB'):
                    lemmatized += tags[j][0].lemmatize('v')
                    lemmatized += ' '
                elif tags[j][1].startswith('JJ'):
                    lemmatized += tags[j][0].lemmatize('a')
                    lemmatized += ' '
                else:
                    lemmatized += tags[j][0]
                    lemmatized += ' '
        to_lemm['lemm_sub'][i] = lemmatized.strip()
    return to_lemm

In [47]:
lemm = lemmatize_2(lemm)
lemm.head()

c:\users\limri\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,id,submission_title,submission_text,author,body,depth,parent_id,score,subreddit_name_prefixed,replies,length,labels,cleaned,cleaned_sub,expand,expand_sub,lemmatized,lemm_sub
0,d4xury6,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",Ryvaeus,"No questions, just wanted to thank you for pos...",0,t3_4r2tl4,31,r/Philippines,4,245,supporting,no questions wanted thank posting topic birth ...,updated september scroll downhi lengthy post c...,no questions wanted thank posting topic birth ...,updated september scroll downhi lengthy post c...,no question want thank post topic birth contro...,updated september scroll downhi lengthy post c...
1,d4yi9y9,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",Somebody_over_there,Holy shit thank you so much! \nAs someone ...,0,t3_4r2tl4,7,r/Philippines,8,393,supporting,holy shit thank much someone recently turned s...,updated september scroll downhi lengthy post c...,holy shit thank much someone recently turned s...,updated september scroll downhi lengthy post c...,holy shit thank much someone recently turn sup...,updated september scroll downhi lengthy post c...
2,d4yhm4h,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",PotatoMurderer,> people there are nice and friendly; super no...,0,t3_4r2tl4,7,r/Philippines,0,507,commenting,people nice friendly super non judgmental espe...,updated september scroll downhi lengthy post c...,people nice friendly super non judgmental espe...,updated september scroll downhi lengthy post c...,people nice friendly super non judgmental espe...,updated september scroll downhi lengthy post c...
3,d4y5kmd,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",mazikeen24,Thanks for posting this! Have you tried Yaz or...,0,t3_4r2tl4,3,r/Philippines,16,238,supporting,thanks posting tried yaz oral contraceptives p...,updated september scroll downhi lengthy post c...,thanks posting tried yaz oral contraceptives p...,updated september scroll downhi lengthy post c...,thanks post tried yaz oral contraceptive prior...,updated september scroll downhi lengthy post c...
4,d4y8tvk,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",a_reverse_giraffe,"As a minor, did you need to show your parents ...",0,t3_4r2tl4,3,r/Philippines,4,55,questioning,minor need show parents consent,updated september scroll downhi lengthy post c...,minor need show parents consent,updated september scroll downhi lengthy post c...,minor need show parent consent,updated september scroll downhi lengthy post c...


In [48]:
def analyze_text(lemm):
    """
    Function for analyzing sentiment and subjectivity based on class jupyter notebook on text analysis
    """
    text = lemm.copy()
    sentiment = []
    subjectivity = []
    for i in text['lemmatized']:
        blob = tb(i)
        sentiment.append(blob.polarity)
        subjectivity.append(blob.subjectivity)
    text['sentiment'] = sentiment
    text['subjectivity'] = subjectivity
    sentiment = []
    subjectivity = []
    for i in text['lemm_sub']:
        blob = tb(i)
        sentiment.append(blob.polarity)
        subjectivity.append(blob.subjectivity)
    text['sentiment_sub'] = sentiment
    text['subjectivity_sub'] = subjectivity
    return text

In [49]:
sent = analyze_text(lemm)
sent.head()

,id,submission_title,submission_text,author,body,depth,parent_id,score,subreddit_name_prefixed,replies,...,cleaned,cleaned_sub,expand,expand_sub,lemmatized,lemm_sub,sentiment,subjectivity,sentiment_sub,subjectivity_sub
0,d4xury6,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",Ryvaeus,"No questions, just wanted to thank you for pos...",0,t3_4r2tl4,31,r/Philippines,4,...,no questions wanted thank posting topic birth ...,updated september scroll downhi lengthy post c...,no questions wanted thank posting topic birth ...,updated september scroll downhi lengthy post c...,no question want thank post topic birth contro...,updated september scroll downhi lengthy post c...,0.333333,0.416667,0.109876,0.533718
1,d4yi9y9,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",Somebody_over_there,Holy shit thank you so much! \nAs someone ...,0,t3_4r2tl4,7,r/Philippines,8,...,holy shit thank much someone recently turned s...,updated september scroll downhi lengthy post c...,holy shit thank much someone recently turned s...,updated september scroll downhi lengthy post c...,holy shit thank much someone recently turn sup...,updated september scroll downhi lengthy post c...,0.104167,0.308333,0.109876,0.533718
2,d4yhm4h,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",PotatoMurderer,> people there are nice and friendly; super no...,0,t3_4r2tl4,7,r/Philippines,0,...,people nice friendly super non judgmental espe...,updated september scroll downhi lengthy post c...,people nice friendly super non judgmental espe...,updated september scroll downhi lengthy post c...,people nice friendly super non judgmental espe...,updated september scroll downhi lengthy post c...,0.189815,0.540741,0.109876,0.533718
3,d4y5kmd,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",mazikeen24,Thanks for posting this! Have you tried Yaz or...,0,t3_4r2tl4,3,r/Philippines,16,...,thanks posting tried yaz oral contraceptives p...,updated september scroll downhi lengthy post c...,thanks posting tried yaz oral contraceptives p...,updated september scroll downhi lengthy post c...,thanks post tried yaz oral contraceptive prior...,updated september scroll downhi lengthy post c...,0.133333,0.133333,0.109876,0.533718
4,d4y8tvk,"I got my contraceptive implant, Implanon, last...","**UPDATED September 26, 2018; scroll down**\n\...",a_reverse_giraffe,"As a minor, did you need to show your parents ...",0,t3_4r2tl4,3,r/Philippines,4,...,minor need show parents consent,updated september scroll downhi lengthy post c...,minor need show parents consent,updated september scroll downhi lengthy post c...,minor need show parent consent,updated september scroll downhi lengthy post c...,-0.050000,0.200000,0.109876,0.533718


In [50]:
new_df = sent[['id','depth','parent_id','score','lemmatized','lemm_sub','sentiment','subjectivity','sentiment_sub','subjectivity_sub','labels']]

In [51]:
new_df.head()

,id,depth,parent_id,score,lemmatized,lemm_sub,sentiment,subjectivity,sentiment_sub,subjectivity_sub,labels
0,d4xury6,0,t3_4r2tl4,31,no question want thank post topic birth contro...,updated september scroll downhi lengthy post c...,0.333333,0.416667,0.109876,0.533718,supporting
1,d4yi9y9,0,t3_4r2tl4,7,holy shit thank much someone recently turn sup...,updated september scroll downhi lengthy post c...,0.104167,0.308333,0.109876,0.533718,supporting
2,d4yhm4h,0,t3_4r2tl4,7,people nice friendly super non judgmental espe...,updated september scroll downhi lengthy post c...,0.189815,0.540741,0.109876,0.533718,commenting
3,d4y5kmd,0,t3_4r2tl4,3,thanks post tried yaz oral contraceptive prior...,updated september scroll downhi lengthy post c...,0.133333,0.133333,0.109876,0.533718,supporting
4,d4y8tvk,0,t3_4r2tl4,3,minor need show parent consent,updated september scroll downhi lengthy post c...,-0.050000,0.200000,0.109876,0.533718,questioning


In [52]:
new_df.to_csv('preprocessed_data_2.csv')